# Deep Q Network

> Credits: This work is based on a number of sources.  But I would like to give credit to Dr Mike Allen (PenCHORD, UoE) who introduced me to the method.

## Imports

### gym

In [1]:
import gym
gym.__version__

'0.21.0'

### pytorch

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

### standard

In addition to the standard data science imports this implementation will also make use of a `dequeue`. This will act as the DQN's memory buffer (used during sampling).  When this is full one sample will enter and the oldest will leave.

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import deque

## Parameters

In [46]:
################ EXPLORATION / EXPLOITATION SETTINGS ###########################

# default value for maximum epsilon to anneal
DEFAULT_EPSILON_MAX = 1.0

EXPLORATION_DECAY = 0.99
EXPLORATION_MIN = 0.1

################################################################################

In [47]:
#################### OPTIMISATION ##############################################

# discount rate of future rewards
GAMMA = 0.99

# replay buffer size
DEFAULT_BUFFER_SIZE = 10_000

# mini batch size for training
DEFAULT_BATCH_SIZE = 5

################################################################################

In [48]:
######################### DQN AGENT ############################################

# default learning rate
DEFAULT_LR = 0.01

# default neurons in middle layer of DQN
DEFAULT_N_NEURONS_ML = 24
################################################################################

## Agent class

In [26]:
class DQN(nn.Module):
    '''
    
    In a Double Deep Q Network this network architecture is used for both the 
    policy network and the target networks.  
    
    Policy network chooses the action.
    Target network returns the Q.
    
    The seperation is done for stability.  Two consecutive states will be 
    close to each other and an update to a single network will change
    nearby values of Q.
    
    Implementation Notes:
    -----
    Objective = MSE
    Optimizer = Adam
    
    The architecture used is a simple 3 layer network.
        
    '''
    def __init__(self, observation_space, action_space,
                 n_neurons_middle=DEFAULT_N_NEURONS_ML,
                 epsilon_init=DEFAULT_EPSILON_MAX, random_seed=None,
                 learning_rate=DEFAULT_LR):
        '''
        Constructor for DQN agent. Set parameters and create network 
        architecture.
        
        Parameters:
        ----------
        observation_space: int
            Size of the observation space
            
        action_space: int
            The number of actions e.g. 5 that an agent could choose.
            
        n_neurons_middle: int, optional (default=DEFAULT_N_NEURONS_ML)
            The number of neurons int he middle layer of the network.
        
        epsilon_init: float, optional (default=DEFAULT_EPSILON_MAX)
            The initial value of epsilon.  This value epsilon 
            controls the exploration rate for the agent.
            
        random_seed: int, optional (default=None)
            controls the exploration sampling. set to an int for repeatable
            exploration (does not control SGD!)
            
        learning_rate: float, optional (default=DEFAULT_LR)
            LR used by the Adam optimiser.
            
        '''
        
        # Inherit parent (nn.module) methods using super init
        super(DQN, self).__init__()
        
        # exploration / exploitation settings
        self.epsilon = epsilon_init
        self.rng = np.random.default_rng(random_seed)
        
        # network architecture
        self.net = nn.Sequential(
            # Linear(no. in features, no. out features)
            nn.Linear(observation_space, n_neurons_middle),
            nn.ReLu(),
            nn.Linear(n_neurons_middle, n_neurons_middle),
            nn.ReLu(),
            nn.Linear(n_neurons_middle, action_space))
        
        
        self.objective = nn.MSELoss()
        self.optimizer = optim.Adam(self.net.parameters, lr=learning_rate)
        
        
    def take_action(self, state):
        '''
        Choose an action from the action space.
        
        Epsilon greedy implementation.  
        
        Parameters:
        ----------
        state: np.ndarray
            observations of the environment.
        
        Returns:
        -------
        int
            The zero indexed action that has been chosen.
        '''
        if self.rng.uniform() < self.epsilon:
            # take range action
            action = rng.integers(0, action_space)
        else:
            # predict Q
            q_values = self.net(torch.FloatTensor(state))
            # max Q
            action = np.argmax(q_values.detach().numpy()[0])
            
        return action
    
    def forward(self, x):
        '''
        Foward pass through the net. Returns a prediction.
        '''
        return self.net(x)

## Replay buffer

In [30]:
class ReplayBuffer:
    '''
    Using a replay buffer helps to stablise the optimisation.  
    Samples appear to be Independent Identically Distributed (IID).
    
    Implementation notes.
    --------------------
    Here the buffer in implemented as a `collections.deque`.  When full the 
    oldest observations are pushed out.
    '''
    def __init__(self, buffer_size=DEFAULT_BUFFER_SIZE, random_seed=None):
        self.buffer = deque(maxlen=buffer_size)
        self.rng = np.random.default_rng(random_seed)
    
    def store_experience(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    @property
    def size(self):
        '''
        No. observations in the replay buffer. Returns int.
        '''
        return len(self.buffer)
        
    def sample(self, size):
        '''
        uniform sample of observations from the replay buffer
        '''
        return rng.choice(self.deque, size=size)

In [43]:
def optimize(policy_net, target_net, replay_buffer, 
             batch_size=DEFAULT_BATCH_SIZE):
    '''
    Parameters:
    -----------
    policy_net: DQN
        Policy network to predict best action (best Q).
    target_net: DQN
        Target network to provide target of Q for the selected next action.
    replay_buffer: ReplayBuffer 
        A memory buffer from which experience is uniformly sampled 
    batch_size: int
        Mini batch size for training.  
        
    '''    
    # return if replay buffer size if less than 
    if replay_buffer.size < batch_size:
        return
    
    # Reduce exploration rate (exploration rate is stored in policy net)
    policy_net.epsilon *= EXPLORATION_DECAY
    policy_net.epsilon = max(EXPLORATION_MIN, policy_net.epsilon)
    
    # sample a mini batch from the replay buffer
    training_batches = replay_buffer.sample(size=batch_size)
    
    # loop through the mini batch
    for state, action, reward, state_next, terminal in training_batches:
        
        state_action_values = policy_net(torch.FloatTensor(state))
                                    
        # Get Q from target network in order to do policy net update
        
        # current state Q predicted by policy net - call this "expected"
        expected_state_action_values = \
            policy_net(torch.FloatTensor(state)).detach()
        
        if not terminal:
            ###################### POLICY NET ##################################
            # get best action for the next state from policy network
            policy_next_state_action_values = \
                policy_net(torch.FloatTensor(state_next)).detach()
            
            best_action = np.argmax(policy_next_state_action_values[0].numpy())
            ####################################################################
            
            ##################### TARGET NET ###################################
            # predict Q with target network and use best action from policy net
            next_state_action_values = \
                target_net(torch.FloatTensor(state_next)).detach()
            
            best_next_q = next_state_action_values[0][best_action].numpy()
            ####################################################################
            
            ################# UPDATE Q WITH BELLMAN EQ. ########################
            best_next_q = reward + (GAMMA * best_next_q)
            expected_state_action_values[0][action] = updated_q
            ####################################################################
            
        else:
            # Set Q for all actions to reward (-1)
            expected_state_action_values[0] = reward
            
        # Reset net gradients
        policy_net.optimizer.zero_grad()  
        # calculate loss
        loss_v = nn.MSELoss()(state_action_values, expected_state_action_values)
        # Backpropogate loss
        loss_v.backward()
        # Update network gradients
        policy_net.optimizer.step()  
        
    return    